<a href="https://colab.research.google.com/github/kaeatitayauk/CE888_2020_DSS/blob/master/Lab5/Recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

#Load the data from ``jester-data-1.csv''

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-7.86,99.00,99.00,

In [0]:
arr = df.copy() 
arr = arr.drop(arr.columns[0], axis=1)

In [0]:
arr = df.values
print(arr.shape)

(24983, 101)


In [0]:
rated = np.where(arr!=99)
rated

(array([    0,     0,     0, ..., 24982, 24982, 24982]),
 array([ 0,  1,  2, ..., 70, 71, 87]))

In [0]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1835357 (1835357,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [0]:
arr[idx[0][0], idx[1][0]]

-0.97

In [0]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 2

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=1000, alpha=0.0001):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)

In [0]:
# caseA1: latent_factors=2, iterations=20, alpha=0.0001
sgd(iterations=20, alpha=0.0001)  # Note how the MSE decreases with the number of iterations

58.080751191983424
31.58595044008887
25.27963647291544
24.70124945150947
24.653761662715038
24.64825348560044
24.64633079267306
24.64441336959885
24.641849341877197
24.638108724360226
24.632391413473588
24.623405445787107
24.609057932863553
24.58597142382676
24.548739553086236
24.48883264230683
24.393120668911024
24.242214956286016
24.00945592746136
23.66261008844112


In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 7.554e+01, -4.803e-02, -6.486e-01, ..., -7.598e-02, -8.285e-01,
         2.464e-01],
       [ 9.530e+01,  2.216e+00,  1.517e+00, ...,  1.979e+00,  1.101e+00,
         2.524e+00],
       [ 5.453e+01,  5.816e-01,  1.639e-01, ...,  5.069e-01, -1.724e-02,
         7.770e-01],
       ...,
       [ 3.869e+01, -3.674e-01, -6.838e-01, ..., -3.513e-01, -7.475e-01,
        -2.070e-01],
       [ 3.723e+01,  8.627e-01,  5.894e-01, ...,  7.704e-01,  4.270e-01,
         9.830e-01],
       [ 7.450e+01,  9.426e-01,  3.758e-01, ...,  8.274e-01,  1.160e-01,
         1.205e+00]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [0]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 75.53943788686308)","(-7.82, -0.048033152786121115)","(8.79, -0.6486024819702265)","(-9.66, -0.7709603863353689)","(-8.16, -2.3546946014484225)","(-7.52, -0.2540743592186344)","(-8.5, 0.3246808738855826)","(-9.85, -1.002957649318921)","(4.17, -0.993342097765229)","(-8.98, -1.4351772161667473)","(-4.76, 0.4438019142224788)","(-8.5, 0.7401334937052227)","(-6.75, 0.3195187169504473)","(-7.18, -2.2429421741137303)","(8.45, 0.3601484709908999)","(-7.18, -2.1728148747039455)","(-7.52, -3.383821415073389)","(-7.43, -1.6120776646874562)","(-9.81, -1.1219726276534838)","(-9.85, -0.6738159241147741)","(-9.85, -1.9227797926986656)","(-9.37, 1.148384813674465)","(1.5, -0.1636573249806222)","(-4.37, -0.9525565760946079)","(-9.81, -2.4382903754671963)","(-8.5, -0.637622793813971)","(1.12, 0.1967816005125064)","(7.82, 2.241084854870151)","(2.86, 0.4134704755162038)","(9.13, 2.196641720628337)","(-7.43, -1.6390881554769765)","(2.14, 1.1390004089342125)","(-4.08, 2.3100166093101504)","(-9.08, -2.275387088236601)","(7.82, -0.2796112803166542)","(5.05, 2.0021597078122517)","(4.95, 2.5554763163600343)","(-9.17, -2.068536409857294)","(-8.4, 0.24549044462475167)","(-8.4, -0.08854650010634565)",...,"(8.59, 1.2512012279153115)","(3.59, 1.9388705252531533)","(-6.84, -1.0371502419601901)","(-9.03, -1.7086233155370734)","(2.82, 1.2772171082232036)","(-1.36, 1.4691594591075041)","(-9.08, -1.7418130649476073)","(8.3, 1.63781643399652)","(5.68, 1.6168991815155445)","(-4.81, -0.7789727518884569)","(99.0, -1.0833321805915819)","(99.0, 1.68550189765961)","(99.0, 0.27806705905831447)","(99.0, -1.7673028752335251)","(99.0, -0.8069140516641322)","(99.0, 1.5669466138905694)","(99.0, -0.12078588824961722)","(-9.42, 0.7914972144023295)","(99.0, -0.729500673510315)","(99.0, 0.1746338680741131)","(99.0, 0.8752710879136427)","(-7.72, 0.12230459532673003)","(99.0, 1.1612660517414042)","(99.0, -0.10939821351450107)","(99.0, 0.05186148789625878)","(99.0, -0.2534067554706888)","(99.0, 0.9183300970692764)","(99.0, 1.0131832013005528)","(99.0, 2.197281410036537)","(99.0, -0.17956004094425237)","(2.82, 1.0006389527017903)","(99.0, 0.2639947100308411)","(99.0, 1.4180402057341368)","(99.0, 0.35019197247697176)","(99.0, 0.2354178923352765)","(99.0, 0.5596159619405137)","(-5.63, 0.6870744225847342)","(99.0, -0.07597731591161493)","(99.0, -0.8285476437633899)","(99.0, 0.2463570049747465)"
1,"(100.0, 95.2991473668757)","(4.08, 2.216145024118809)","(-0.29, 1.5170395517654167)","(6.36, 1.7964699344675916)","(4.37, -0.029671829005687867)","(-2.38, 1.4139150610157643)","(-9.66, 3.1747563221213637)","(-0.73, 0.41599073854174545)","(-5.34, -0.042314886750134256)","(8.88, 0.5626193317543003)","(9.22, 2.4318533936970637)","(6.75, 3.3224843829292605)","(8.64, 3.1366390541840112)","(4.42, -1.4358778629975322)","(7.43, 2.936887698186028)","(4.56, -1.4159599581709603)","(-0.97, -3.1510085211815544)","(4.66, -0.5425936808430716)","(-0.68, -0.11740143832889656)","(3.3, 1.294620772320273)","(-1.21, 0.0806368388076551)","(0.87, 3.8872988068097065)","(8.64, 2.3863165587737716)","(8.35, 1.5595380160006778)","(9.17, -0.5572623313109719)","(0.05, 1.8104182649146427)","(7.57, 2.8850962995740685)","(4.71, 4.975905494312184)","(0.87, 3.128772298153664)","(-0.39, 4.607487458552507)","(6.99, 1.077797016526286)","(6.5, 4.038205278881527)","(-0.92, 4.83100581606379)","(7.14, -0.034337808518373106)","(9.03, 2.598193683963209)","(-1.8, 4.895418119524677)","(0.73, 4.889494697726369)","(7.09, -0.34254198559286664)","(3.4, 2.8019357341176563)","(-0.87, 2.8159306095709384)",...,"(-6.7, 4.463517505507412)","(-3.35, 4.848843063371697)","(-9.03, 1.8511504758403252)","(4.47, 0.8110155695413348)","(4.08, 4.126277081199909)","(-3.83, 4.375822076630806)","(8.74, 0.39381199961237

In [0]:
# Calculate the performance (e.g., MSE) of the algorithm on the validation dataset
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseA1=sum(error)/len(error)
print("caseA1: ",mseA1)

caseA1:  23.46451095898704


Repeat the two points above changing hyper-parameters (i.e., learning rate, number of iterations of SVD, number of latent factors, etc.) as needed to get good results (you can create multiple validation sets if you want, and run a bootstrap!)

In [0]:
# caseA2: latent_factors=2, iterations=30, alpha=0.0001
n_latent_factors = 2
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=15,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseA2=sum(error)/len(error)
print("caseA2: ",mseA2)

60.08308363269507
32.04462327950698
25.47211675257234
24.70675763791263
24.625315851621966
24.605428335684888
24.592118895773186
24.58004165917319
24.56851792873112
24.557359036421065
24.546409630347622
24.535467731452762
24.524241401816106
24.512294630057138
24.4989664068679
caseA2:  40.756220788200714


In [0]:
# caseA2: latent_factors=2, iterations=30, alpha=0.0001
n_latent_factors = 2
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=30,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseA2=sum(error)/len(error)
print("caseA2: ",mseA2)

60.08308363269507
32.04462327950698
25.47211675257234
24.70675763791263
24.625315851621966
24.605428335684888
24.592118895773186
24.58004165917319
24.56851792873112
24.557359036421065
24.546409630347622
24.535467731452762
24.524241401816106
24.512294630057138
24.4989664068679
24.48324740759913
24.46359301652417
24.43764329268763
24.40181216397507
24.35070535311961
24.276344051566312
24.167242293188018
24.00757271627089
23.777043350751207
23.452719716822028
23.014539127493418
22.45555268775852
21.794285823646693
21.080567721407995
20.384754755310084
caseA2:  43.994942460846055


In [0]:
# caseA3: latent_factors=2, iterations=40, alpha=0.0001
n_latent_factors = 2
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=40,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseA3=sum(error)/len(error)
print("caseA3: ",mseA3)

60.08308363269507
32.04462327950698
25.47211675257234
24.70675763791263
24.625315851621966
24.605428335684888
24.592118895773186
24.58004165917319
24.56851792873112
24.557359036421065
24.546409630347622
24.535467731452762
24.524241401816106
24.512294630057138
24.4989664068679
24.48324740759913
24.46359301652417
24.43764329268763
24.40181216397507
24.35070535311961
24.276344051566312
24.167242293188018
24.00757271627089
23.777043350751207
23.452719716822028
23.014539127493418
22.45555268775852
21.794285823646693
21.080567721407995
20.384754755310084
19.771779648504907
19.277668969130296
18.90501376713214
18.635124571008845
18.443025708379164
18.30641905471917
18.208437397187435
18.1372392120148
18.084743234509684
18.0454600335474
caseA3:  41.794925595306246


In [0]:
# caseB1: latent_factors=3, iterations=20, alpha=0.0001
n_latent_factors = 3
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=40,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseB1=sum(error)/len(error)
print("caseB1: ",mseB1)

52.945743551915676
31.124553515077427
26.02945728402816
24.893879005275327
24.665215064291917
24.60973994947886
24.587033497689415
24.56965764680724
24.551187573717534
24.528775650845184
24.499542941425503
24.459465684050823
24.40259672386842
24.320197133500297
24.19976881646538
24.024269766180367
23.772257970972607
23.420372676492466
22.94996489813342
22.35846426530948
21.671560416788974
20.946191615383828
20.255308268706226
19.659927998610378
19.18855502590653
18.837383012844768
18.58469177943708
18.405165395742767
18.27733844964749
18.185349729373307
18.11816540094374
18.068266435255058
18.030532671508293
18.001447984028765
17.978567582626308
17.960166863111372
17.94500964785377
17.932194026947272
17.92104879028901
17.911063093750887
caseB1:  34.06542638065412
52.905451462647875
31.14221953548958
26.031823275239883
24.887501648274934
24.656456919931188
24.60037566385978
24.577557475952307
24.560279792443076
24.542104465353344
24.520263981096516
24.492022761889704
24.4535746126051
24

In [0]:
# caseB2: latent_factors=4, iterations=30, alpha=0.0001
n_latent_factors = 4
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=40,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseB2=sum(error)/len(error)
print("caseB2: ",mseB2)

48.844811971355185
30.863069492129544
26.559862565545966
25.176414747121832
24.767078555285046
24.64197581579918
24.598182876062385
24.5776120872905
24.563368599792547
24.550079673697628
24.535432142169405
24.51764194402052
24.494528559283772
24.462951953270785
24.41823010924215
24.353405419606826
24.25834259567731
24.118796199528195
23.91589969377684
23.627077456898544
23.23002257972207
22.711320136076296
22.078854336386815
21.371470427929925
20.65496954317758
20.00011758683685
19.4555052656586
19.035227825943956
18.72646893091008
18.505084286700576
18.347172116639012
18.233768943602335
18.15125093154026
18.09021995009736
18.04425996861649
18.008972323794335
17.98130899355466
17.95912907100101
17.940905845971223
17.92553241484083
caseB2:  33.00232837162642
48.85060640878989
30.85363350560991
26.555606246737987
25.171664878538362
24.76152831127929
24.635932354344707
24.591852625971583
24.571123487230015
24.556837097204692
24.543651039315527
24.52931928707761
24.512169144328823
24.49019

In [0]:
# caseB3: latent_factors=5, iterations=40, alpha=0.0001
n_latent_factors = 5
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=40,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseB3=sum(error)/len(error)
print("caseB3: ",mseB3)

45.654937916680325
30.584843017506003
26.944311137953157
25.476397973657424
24.91867971112305
24.70673161804132
24.62255510314717
24.585092582420813
24.56438765075661
24.549091732861402
24.534456229391505
24.517857677044777
24.49701611407576
24.4691175417444
24.43016523481288
24.374292340380766
24.292941705991097
24.173961080792832
24.00089794902721
23.75320555597863
23.408669342965624
22.949687389821392
22.373849540358563
21.70514222554064
20.996706968153088
20.317051843747958
19.72455710945649
19.248162552434056
18.88664538163574
18.621207079837298
18.428656485628498
18.288663233954605
18.185774200298177
18.108948677211217
18.05046555419034
18.00493274794431
17.96855484313346
17.938627208045258
17.913193239512346
17.890812319972216
caseB3:  31.791567599777757
45.64942513797693
30.602993991482624
26.946290176426928
25.469506489891504
24.90776644521071
24.694017127862708
24.6089492410422
24.57090153594762
24.549622876852354
24.533579978693744
24.517850933303027
24.499591784201115
24.47